# Black Jack <br>
<font size= +1>
<p style="text-align: justify">
Se intentará resolver a continuación una versión simplificada del famoso juego BlackJack o 21. Para esto, se tienen las siguientes reglas:<br>

1. Solamente es un jugador y el Dealer, y en cada juagada se apuesta una cantidad fija de dinero (se requiere una política por jugada y los juegos son independientes)<br>

2. Se asume un mazo infinito (todas las cartas tienen la misma probabilidad de salir)<br>

3. El jugador juega primero, y solo tiene dos jugadas posibles: parar o pedir otra carta.<br>

4. El jugador puede pedir cartas hasta que: llegue a 21 puntos (gana automáticamente), se pase de 21 puntos (pierde automáticamente), tenga 4 cartas o más (gana automáticamente) o decida parar.<br>

5. El Dealer empieza con una sola carta y una vez que el jugador decide parar, y no haya perdido o ganado automáticamente.<br>

6. El Dealer tiene que pedir_carta siempre que tenga menos de 17 puntos.<br>

7. El Dealer tiene que parar si tiene 17 puntos o más.<br>

8. Si el Dealer tiene 4 cartas, o un número mayor que el jugador (y menos que 21) gana. De otra forma, gana el jugador.
</p>


</font>

In [3]:
using Plots
using NBInclude
@nbinclude("../algoritmos_reforzamiento.ipynb")


q_learning (generic function with 2 methods)

In [4]:
struct BJ_s <:Estado
    d #valor de primer carta del dealer
    sum #suma del jugador hasta el momento
    n #numero de cartas del jugador
    as #indica si se puede sumar 10 a sum o no (1,0), es decir, si tiene un As.

    function BJ_s(d::Int,sum::Int,n::Int,as::Int)
        if (d > 10 || sum>22 || n>4 || as>1 )
            error("estado invalido")
        else
            return new(d,sum,n,as)
        end
    end
end

In [5]:
struct BJ_a <:Accion # Se puede pedir otra carta o finalizar el juego.
    a
    function BJ_a(a)
        if (a != :fin && a!= :otra)
            error("accion invalida")
        else
            new(a)
        end
    end
end

In [6]:
struct BJ <: MDP
    states
    actions
    function BJ(states::Array{BJ_s,1}, actions::Array{BJ_a,1})
        new(states,actions)
    end
end

In [7]:
#terminal
function terminal(mdp::BJ,s::BJ_s)
    (s.d==s.sum==s.n==0) && (s.as==1 || s.as==-1) ? true : false
end


#acciones legales
function aLegales(mdp::BJ,s::BJ_s)
    if terminal(mdp,s)
        return nothing
    end
    
    acciones = []
    
    if s.sum<21 && s.n<4
        push!(acciones,BJ_a(:otra))
    end
    
    push!(acciones,BJ_a(:fin))
    
    acciones
end
                        
#recompensa
function r(mdp::BJ,s::BJ_s,a::BJ_a,s2::BJ_s)
    if terminal(mdp,s) return r(mdp,s) end
    0
end

function r(mdp::BJ,s::BJ_s)
    if s.d==s.sum==s.n==0 && s.as==1
        return 1
    elseif s.d==s.sum==s.n==0 && s.as==-1
        return -1
    end
    0
end


r (generic function with 2 methods)

In [8]:
est=[BJ_s(1,1,1,0)]
pop!(est)

for k in 2:4 #num
    if k==1
        x=10 
    elseif k==2 
        x=20
    else
        x=21
    end
    for i in 1:10 #dealer
        for j in 1:x #sum
            for l in 0:1 #as
                if j>k
                    #if l==1 && j<=(k-1)*10
                        push!(est,BJ_s(i,j,k,l))
                    #elseif l==0
                    #    push!(est,BJ_s(i,j,k,l))
                    #end
                end
            end
        end
    end
end

push!(est,BJ_s(0,0,0,1))
push!(est,BJ_s(0,0,0,-1))

1062-element Array{BJ_s,1}:
 BJ_s(1, 3, 2, 0)  
 BJ_s(1, 3, 2, 1)  
 BJ_s(1, 4, 2, 0)  
 BJ_s(1, 4, 2, 1)  
 BJ_s(1, 5, 2, 0)  
 BJ_s(1, 5, 2, 1)  
 BJ_s(1, 6, 2, 0)  
 BJ_s(1, 6, 2, 1)  
 BJ_s(1, 7, 2, 0)  
 BJ_s(1, 7, 2, 1)  
 BJ_s(1, 8, 2, 0)  
 BJ_s(1, 8, 2, 1)  
 BJ_s(1, 9, 2, 0)  
 ⋮                 
 BJ_s(10, 17, 4, 0)
 BJ_s(10, 17, 4, 1)
 BJ_s(10, 18, 4, 0)
 BJ_s(10, 18, 4, 1)
 BJ_s(10, 19, 4, 0)
 BJ_s(10, 19, 4, 1)
 BJ_s(10, 20, 4, 0)
 BJ_s(10, 20, 4, 1)
 BJ_s(10, 21, 4, 0)
 BJ_s(10, 21, 4, 1)
 BJ_s(0, 0, 0, 1)  
 BJ_s(0, 0, 0, -1) 

In [9]:
ac = [BJ_a(:fin), BJ_a(:otra)]
ac

2-element Array{BJ_a,1}:
 BJ_a(:fin) 
 BJ_a(:otra)

In [10]:
BlackJ = BJ(est,ac)

BJ(BJ_s[BJ_s(1, 3, 2, 0), BJ_s(1, 3, 2, 1), BJ_s(1, 4, 2, 0), BJ_s(1, 4, 2, 1), BJ_s(1, 5, 2, 0), BJ_s(1, 5, 2, 1), BJ_s(1, 6, 2, 0), BJ_s(1, 6, 2, 1), BJ_s(1, 7, 2, 0), BJ_s(1, 7, 2, 1)  …  BJ_s(10, 18, 4, 0), BJ_s(10, 18, 4, 1), BJ_s(10, 19, 4, 0), BJ_s(10, 19, 4, 1), BJ_s(10, 20, 4, 0), BJ_s(10, 20, 4, 1), BJ_s(10, 21, 4, 0), BJ_s(10, 21, 4, 1), BJ_s(0, 0, 0, 1), BJ_s(0, 0, 0, -1)], BJ_a[BJ_a(:fin), BJ_a(:otra)])

In [11]:
function dealer(d::Int64)
    
    n=1
    total=d
    while n<4
        x=rand([1,2,3,4,5,6,7,8,9,10,10,10,10])
        total+=x
        
        if x==1 && 16>total+10<=21 total+=10 end
        
        if total + x >=17 break end
        n+=1
    end
    
    total,n
end

dealer (generic function with 1 method)

In [13]:
function step(mdp::BJ,s::BJ_s,a ::BJ_a)
    
    
    if a.a==:fin
        
        (s.as==1 && s.sum+10<=21) ? sum=s.sum+10 : sum=s.sum
        if sum==21 || (s.n==4 && sum<=21) return BJ_s(0,0,0,1)
        else 
            d,dn = dealer(s.d)
            if d>21 return BJ_s(0,0,0,1) end
            if dn==4 && d<=21 return BJ_s(0,0,0,-1) end
            (sum >= d) ? BJ_s(0,0,0,1) : BJ_s(0,0,0,-1)
        end
    else #a.a==:otra
        x=rand([1,2,3,4,5,6,7,8,9,10,10,10,10])
        
        if s.sum+x > 21 return BJ_s(0,0,0,-1) end
        
        if x==1 return BJ_s(s.d,s.sum+x,s.n+1,1) end
        
        return BJ_s(s.d,s.sum+x,s.n+1,s.as)
    end
        
end

step (generic function with 1 method)

In [22]:
function resultado(mdp::BJ, episodes,ϵ,α, runs)

    s=BJ_s(7,10,2,0)

    q_value=Dict((s,rand(aLegales(mdp,s))) =>0.0)

    for s ∈ mdp.states
        if terminal(mdp,s) continue end
        for a ∈ aLegales(mdp,s)
            q_value[(s,a)]=0.0
        end
    end
    #q_value = Dict( (s,a)=> 0 for s in mdp.states, a in aLegales(s))
    start=rand(mdp.states)


    π_ ,steps, rewards = sarsa(mdp, q_value, start , ϵ, α, episodes,true, runs)

    
    println(rand(π_,30))
end

resultado (generic function with 2 methods)

In [23]:
episodes = 1000000
ϵ= 0.5
α=0.5
runs=1
resultado(BlackJ, episodes,ϵ,α, runs)

Pair{BJ_s,BJ_a}[BJ_s(6, 11, 2, 0)=>BJ_a(:otra), BJ_s(9, 11, 2, 0)=>BJ_a(:otra), BJ_s(2, 19, 3, 1)=>BJ_a(:fin), BJ_s(1, 16, 3, 1)=>BJ_a(:otra), BJ_s(2, 11, 4, 0)=>BJ_a(:fin), BJ_s(10, 15, 4, 1)=>BJ_a(:fin), BJ_s(8, 20, 3, 1)=>BJ_a(:fin), BJ_s(4, 8, 4, 0)=>BJ_a(:fin), BJ_s(4, 7, 3, 1)=>BJ_a(:otra), BJ_s(2, 18, 3, 0)=>BJ_a(:fin), BJ_s(9, 20, 2, 1)=>BJ_a(:fin), BJ_s(9, 20, 2, 0)=>BJ_a(:fin), BJ_s(6, 10, 4, 0)=>BJ_a(:fin), BJ_s(8, 12, 4, 1)=>BJ_a(:fin), BJ_s(2, 9, 4, 0)=>BJ_a(:fin), BJ_s(9, 19, 2, 1)=>BJ_a(:fin), BJ_s(10, 8, 3, 1)=>BJ_a(:otra), BJ_s(10, 12, 4, 0)=>BJ_a(:fin), BJ_s(2, 5, 4, 1)=>BJ_a(:fin), BJ_s(3, 14, 4, 0)=>BJ_a(:fin), BJ_s(2, 13, 3, 1)=>BJ_a(:fin), BJ_s(2, 19, 4, 1)=>BJ_a(:fin), BJ_s(3, 6, 2, 0)=>BJ_a(:otra), BJ_s(3, 9, 3, 0)=>BJ_a(:otra), BJ_s(4, 5, 4, 1)=>BJ_a(:fin), BJ_s(1, 20, 3, 1)=>BJ_a(:fin), BJ_s(8, 7, 4, 1)=>BJ_a(:fin), BJ_s(1, 14, 2, 1)=>BJ_a(:fin), BJ_s(1, 9, 4, 1)=>BJ_a(:fin), BJ_s(4, 3, 2, 1)=>BJ_a(:otra)]
